In [1]:
import spacy
import corenlp
import nltk
from nltk import ngrams
from nltk.tokenize import sent_tokenize,wordpunct_tokenize,word_tokenize,RegexpTokenizer,TweetTokenizer
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import sklearn
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import os,math,glob,re

from collections import Counter
import re as regex
import contractions
import copy

from itertools import chain

nlp = spacy.load('en_core_web_sm')
nlp.max_length=9999999



### Pre-Processing

In [3]:
def contract(text):
    return contractions.fix(text)

def regTokenize(text):
    tok=RegexpTokenizer('[A-Za-z0-9]*[.]?\w+')
    return tok.tokenize(text) 

def lowercase(text):
    return text.lower()

def lemma(words):
    for i in range(0,len(words)):
        words[i]=WordNetLemmatizer().lemmatize(words[i])
    return words

def stemming(words):
    porter_stemmer=PorterStemmer()
    for i in range(0,len(words)):
        words[i]=porter_stemmer.stem(words[i])
    return words

def tweet(words):
    tok=TweetTokenizer()
    return tok.tokenize(words)

def comma(text):
    text = "".join(c for c in text if c not in ('!','.',':',',','"','?','(',')'))
    return text

def getBasicNorm(text):
    for i in range(0,len(text)):
        text[i]=contract(text[i])
        text[i]=lowercase(text[i])
    return text

### Feature Extraction functions

In [4]:
def getWholeString(data):
    string=''
    for i in data:
        string+=i[0]+' '
    return string

def getNGrams(corpusSentence):
    
    corpusSentence=contract(corpusSentence)
    corpusSentence=lowercase(corpusSentence)
    tokenized=regTokenize(corpusSentence)
    
    uni=list(ngrams(tokenized,1))
    bigram=list(ngrams(tokenized,2))
    trigram=list(ngrams(tokenized,3))

    print('uni:',len(uni),' bi:',len(bigram),'tri:',len(trigram))
    grams=[uni,bigram,trigram]
    
    return grams


def postagging(corpusSentence):
    
    corpusSentence=contract(corpusSentence)
    corpusSentence=lowercase(corpusSentence)
#     tokenized=regTokenize(corpusSentence)
    
    doc = nlp(corpusSentence)
#     for i in range(0,5):
#         print(doc[i].pos_,doc[i].lemma_)
    
    return doc

def posPattern(corpusSentence):
    doc = nlp(corpusSentence)
    return doc


# def namedEntities(corpusSentence):
#     corpusSentence=contract(corpusSentence)
#     corpusSentence=lowercase(corpusSentence)
# #     tokenized=regTokenize(corpusSentence)
    
#     doc = nlp(corpusSentence)
# #     for i in range(0,5):
# #         print(doc[i].pos_,doc[i].lemma_)
#     return doc


# -----------------------------------------POS Pattern-----------------------------------------
def getPosPattern(data_sentences):
    sentences_pos=[]
    for i in data_sentences:
        sent=[]
        s=postagging(i)
        for j in s:
            sent.append(j.pos_)
        sentences_pos.append(sent)
    return sentences_pos

def posGrams(sentence):
    
    trigram=list(ngrams(sentence,3))
    fourgram=list(ngrams(sentence,4))
    
#     print('uni:',len(uni),' bi:',len(bigram),'tri:',len(trigram))
    grams=[trigram,fourgram]
    
    return grams
   
def get34grams(sentences_pos):
    
    three_four_grams=[]
    
    for sent_pos in sentences_pos:
        temp=posGrams(sent_pos)
        temp = list(chain(*temp))
#         print(temp)
        three_four_grams.append(temp)
    
    return three_four_grams

### Dataset Loading

In [5]:
dataset=joblib.load('dataset_with_labels.sav')

print(len(dataset),' sentences')

# print(np.array(dataset)[:,0])
X=np.array(dataset)[:,0] # sentences
Y=np.array(dataset)[:,1] # labels
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

print(len(X_train),' ',len(X_test),' ',len(y_train),' ',len(y_test))

11112  sentences
8889   2223   8889   2223


In [6]:
dataset_fact=[]
dataset_nonfact=[]

for i in dataset:
    if(i[1]==1):
        dataset_fact.append(i)
    else:
        dataset_nonfact.append(i)

print(len(dataset_fact),' ' ,len(dataset_nonfact))

4989   6123


### vector loading